# Mezcla binaria (NP's y Mesógenos)

In [1]:
#-----Importa los paquetenes necesartios para correr la simulación

from __future__ import division
import hoomd
import hoomd.md

#-----Define algunas variables relevantes

t_max = 5.0;
p_0   = 0.7; 
p_1   = 0.9;
steps_ramp = 1e5;

#-----Crea un contexto de simulación

hoomd.context.initialize("--mode=cpu", memory_traceback = True);

#-----Extrae la configuración del centro de masa de las partículas

system = hoomd.init.read_gsd("T_CM&NP_5.0_P_0.7_ramp.gsd", frame = -1);

#-----Define cada mesógeno en el marco de referencia local del cuerpo.

rigid = hoomd.md.constrain.rigid();
rigid.set_param('M', 
                types = ['A']*8,
                positions = [(-4,0,0),(-3,0,0),(-2,0,0),(-1,0,0),
                           (1,0,0),(2,0,0),(3,0,0),(4,0,0)]);

#-----Crea los mesógenos.

rigid.create_bodies();

#-----Define la energía potencial.

nl = hoomd.md.nlist.tree();

lj = hoomd.md.pair.lj(r_cut = 3.5, nlist = nl)
lj.set_params(mode = 'shift')

#-----Declara las interacciones entre elementos de la mezcla.

lj.pair_coeff.set('NP','NP', epsilon = 1.0, sigma = 5.0);
lj.pair_coeff.set('M' ,'M' , epsilon = 1.0, sigma = 1.0);
lj.pair_coeff.set('A' ,'A' , epsilon = 1.0, sigma = 1.0);
lj.pair_coeff.set('M' ,'A' , epsilon = 1.0, sigma = 1.0);
lj.pair_coeff.set('NP','M' , epsilon = 1.0, sigma = 3.0);
lj.pair_coeff.set('NP','A' , epsilon = 1.0, sigma = 3.0);

#-----Selecciona un integrador estándar.

hoomd.md.integrate.mode_standard(dt = 0.005);

#------Define dos grupos y efectua su unión.

nanoparticles = hoomd.group.type(name = 'Nano_Particles', type = 'NP');
mesogens = hoomd.group.rigid_center();
groupNP_mes = hoomd.group.union(name = "NP_Mes", a = nanoparticles, b = mesogens);

#----Integra usando un NPT.

press = hoomd.variant.linear_interp(points = [(0,p_0), (steps_ramp, p_1)]);

npt = hoomd.md.integrate.npt(group = groupNP_mes, kT = t_max, tau = 0.5, tauP = 1.0, P = press);
npt.randomize_velocities(seed = 42)

#-----Guarda los datos en archivos .log y .gsd

log_file = "T_" + str(t_max) + "_P_" + str(p_1) + "_ramp.log"
gsd_file = "T_" + str(t_max) + "_P_" + str(p_1) + "_ramp.gsd" 
meso_gsd_file = "T_CM&NP_" + str(t_max) + "_P_" + str(p_1) + "_ramp.gsd"

log = hoomd.analyze.log(filename = log_file,
                         quantities = ['num_particles',
                                     'ndof',
                                     'translational_ndof',
                                     'rotational_ndof',
                                     'potential_energy',
                                     'kinetic_energy',
                                     'translational_kinetic_energy',
                                     'rotational_kinetic_energy',
                                     'temperature',
                                     'pressure',
                                      'volume'],
                         period = 1e3,
                         overwrite = True);
gsd = hoomd.dump.gsd(gsd_file,
               period = 1e3,
               group = hoomd.group.all(),
               overwrite = True); 
meso_gsd = hoomd.dump.gsd(meso_gsd_file,
               period = 1e3,
               group = groupNP_mes,
               overwrite = True); 

HOOMD-blue 2.6.0 DOUBLE HPMC_MIXED TBB SSE SSE2 SSE3 
Compiled: 05/29/2019
Copyright (c) 2009-2019 The Regents of the University of Michigan.
-----
You are using HOOMD-blue. Please cite the following:
* J A Anderson, C D Lorenz, and A Travesset. "General purpose molecular dynamics
  simulations fully implemented on graphics processing units", Journal of
  Computational Physics 227 (2008) 5342--5359
* J Glaser, T D Nguyen, J A Anderson, P Liu, F Spiga, J A Millan, D C Morse, and
  S C Glotzer. "Strong scaling of general-purpose molecular dynamics simulations
  on GPUs", Computer Physics Communications 192 (2015) 97--107
-----
HOOMD-blue is running on the CPU
notice(2): Group "all" created containing 1004 particles
notice(2): constrain.rigid(): Creating 1000 rigid bodies (adding 8000 particles)
-----
You are using tree neighbor lists. Please cite the following:
* M P Howard, J A Anderson, A Nikoubashman, S C Glotzer, and A Z
  Panagiotopoulos. "Efficient neighbor list calculation for mol

In [2]:
hoomd.run(steps_ramp)

notice(2): -- Neighborlist exclusion statistics -- :
notice(2): Particles with 0 exclusions             : 9004
notice(2): Neighbors included by diameter          : no
notice(2): Neighbors excluded when in the same body: yes
** starting run **
Time 00:00:10 | Step 198258 / 298000 | TPS 25.7615 | ETA 01:04:31
Time 00:00:20 | Step 198542 / 298000 | TPS 28.2636 | ETA 00:58:38
Time 00:00:30 | Step 198823 / 298000 | TPS 28.0745 | ETA 00:58:52
Time 00:00:40 | Step 199101 / 298000 | TPS 27.7805 | ETA 00:59:20
Time 00:00:50 | Step 199388 / 298000 | TPS 28.6786 | ETA 00:57:18
Time 00:01:00 | Step 199670 / 298000 | TPS 28.1673 | ETA 00:58:10
Time 00:01:10 | Step 199952 / 298000 | TPS 28.1552 | ETA 00:58:02
Time 00:01:20 | Step 200225 / 298000 | TPS 27.2654 | ETA 00:59:46
Time 00:01:30 | Step 200501 / 298000 | TPS 27.4171 | ETA 00:59:16
Time 00:01:40 | Step 200785 / 298000 | TPS 28.2591 | ETA 00:57:20
Time 00:01:50 | Step 201067 / 298000 | TPS 28.1831 | ETA 00:57:19
Time 00:02:00 | Step 201340 / 2

In [3]:
system.get_metadata()

OrderedDict([('box', <hoomd.data.boxdim at 0x103c1d7f0>),
             ('particles', <hoomd.data.particle_data at 0x10892cd30>),
             ('number_density', 0.49119454679197694),
             ('bonds', <hoomd.data.bond_data at 0x10892cdd8>),
             ('angles', <hoomd.data.angle_data at 0x10892ce48>),
             ('dihedrals', <hoomd.data.dihedral_data at 0x10892ceb8>),
             ('impropers', <hoomd.data.dihedral_data at 0x10892cf28>),
             ('constraints', <hoomd.data.constraint_data at 0x10892cf98>),
             ('pairs', <hoomd.data.bond_data at 0x1089d3048>),
             ('timestep', 298000)])

In [1]:
print(1004 / 0.49119)

2044.0155540625826
